In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [86]:
# Limpiar la columna parameter (nombre contaminante)
def clean_parameter_col(df):
    df['parameter'] = df['parameter'].apply(lambda s: s.strip())
    return df

def preprocess(df):
    df = clean_parameter_col(df)
    df = df[['date', 'parameter', 'SO2', 'SO2_b']]
    df.columns = ['date', 'parameter', 'Value', 'Flag']
    return df

def pivot(df):
    piv = df.pivot(
        columns='parameter',
        index='date',
        values='Value'
    )
    return piv

invalid_flags = set(['P', 'p', 'c', 'd', 'b', 'm', 'l', 'z',
                 'o', 's', 'f', 'e', 'a', 's', 'f', 'h'])

def rm_invalid_flags(df):
    df.loc[df['Flag'].isin(invalid_flags), 'Value'] = np.nan
    return df

In [87]:
DATA_DIR = "../data"
DATA_RAW =  DATA_DIR + "/raw"
DATA_PROCESSED =  DATA_DIR + "/processed"

RAW_CONT_PATH = f"{DATA_RAW}/SD_TecMTY_contaminantes_2021_2022.csv"
RAW_METEO_PATH = f"{DATA_RAW}/SD_TecMTY_meteorologia_2021_2022.csv"

PROCESSED_CONT_PATH = f"{DATA_PROCESSED}/cont.csv"
PROCESSED_MET_PATH = f"{DATA_PROCESSED}/meteo.csv"

PROCESSED_PIV_CONT_PATH = f"{DATA_PROCESSED}/piv_cont.csv"
PROCESSED_PIV_MET_PATH = f"{DATA_PROCESSED}/piv_meteo.csv"

# Contaminantes

In [248]:
cont = pd.read_csv(RAW_CONT_PATH, index_col=0)
cont.head()

,date,parameter,SE,SE_b,NE,NE_b,CE,CE_b,NO,NO_b,...,SO2,SO2_b,SE3,SE3_b,SUR,SUR_b,Norte2,NTE2_b,NE3,NE3_b
205739,2021-07-01 00:00:00,PM10,NaN,x,NaN,x,NaN,x,NaN,x,...,NaN,x,NaN,l,NaN,l,NaN,l,NaN,x
205740,2021-07-01 00:00:00,PM2.5,NaN,x,NaN,x,NaN,x,NaN,x,...,NaN,x,NaN,l,NaN,l,NaN,l,NaN,x
205741,2021-07-01 00:00:00,O3,NaN,x,NaN,x,NaN,x,NaN,x,...,NaN,x,NaN,l,NaN,l,NaN,l,NaN,x
205742,2021-07-01 00:00:00,SO2,NaN,x,NaN,x,NaN,x,NaN,x,...,NaN,x,NaN,l,NaN,l,NaN,l,NaN,x
205743,2021-07-01 00:00:00,NO2,NaN,e,NaN,x,NaN,x,NaN,x,...,NaN,e,NaN,e,NaN,e,NaN,e,NaN,e


In [249]:
cont = preprocess(cont)

Cuantas banderas invalidas hay?
* De estas, cuantas tienen NaN?

In [250]:
tmp = cont[cont['Flag'].isin(invalid_flags)]
print(f"N. filas con bandera invalida: {len(tmp)}")
print(f"N. filas con bandera invalida y con Nan: {tmp['Value'].isna().sum()}")

N. filas con bandera invalida: 1094
N. filas con bandera invalida y con Nan: 1094


* Todas las banderas invalidas tienen valores NaN.

Encontrar el número de NaN con bandera valida

In [262]:
tmp = cont[~cont['Flag'].isin(invalid_flags)]
print(f"Total NaN de banderas validas \n{tmp.isna().sum().sum()}")

Total NaN de banderas validas 
1013


* En total existen 2107 valores NaN, 1094 son banderas invalidas, 1013 son banderas validas.

Guardar df procesada antes de pivotear

In [225]:
cont.to_csv(PROCESSED_CONT_PATH, index=False)

Guardar df pivoteada

In [170]:
piv_cont = pivot(cont)
piv_cont = piv_cont.reset_index()
piv_cont.to_csv(PROCESSED_PIV_CONT_PATH, index=False)
print(len(piv_cont))
piv_cont.head()

10959


parameter,date,CO,NO2,O3,PM10,PM2.5,SO2
0,2021-07-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-07-01 01:00:00,0.49,3.9,22.0,21.0,NaN,2.8
2,2021-07-01 02:00:00,0.51,8.4,17.0,24.0,3.0,3.1
3,2021-07-01 03:00:00,0.50,6.4,19.0,22.0,5.0,3.0
4,2021-07-01 04:00:00,0.51,10.0,14.0,35.0,15.0,3.2


# Meteorologica

In [281]:
met = pd.read_csv(RAW_METEO_PATH, index_col=0)
met.head()

,date,parameter,SE,SE_b,NE,NE_b,CE,CE_b,NO,NO_b,...,SO2,SO2_b,SE3,SE3_b,SUR,SUR_b,Norte2,NTE2_b,NE3,NE3_b
239459,2021-07-01 00:00:00,TOUT,NaN,x,NaN,x,NaN,x,NaN,x,...,NaN,x,NaN,x,NaN,x,NaN,x,NaN,x
239460,2021-07-01 00:00:00,RH,NaN,x,NaN,x,NaN,x,NaN,x,...,NaN,x,NaN,l,NaN,l,NaN,l,NaN,x
239461,2021-07-01 00:00:00,SR,NaN,x,0.0,l,0.0,l,0.0,l,...,0.0,l,0.0,l,0.0,l,0.0,l,0.0,x
239462,2021-07-01 00:00:00,PRS,NaN,x,NaN,x,NaN,x,NaN,x,...,NaN,x,NaN,l,NaN,l,NaN,l,NaN,x
239463,2021-07-01 00:00:00,RAINF,NaN,x,NaN,x,NaN,x,NaN,x,...,NaN,x,NaN,l,NaN,l,NaN,l,NaN,x


In [282]:
met = preprocess(met)

In [283]:
tmp = met[met['Flag'].isin(invalid_flags)]
print(f"N. filas con bandera invalida: {len(tmp)}")
print(f"N. filas con bandera invalida y con Nan: {tmp['Value'].isna().sum()}")

N. filas con bandera invalida: 5165
N. filas con bandera invalida y con Nan: 29


* De las 5165 banderas invalidas, solo 29 de estas son NaN

In [284]:
tmp['Value'].value_counts()

0.0    5136
Name: Value, dtype: int64

* Alrededor de 5136 valores tienen 0.0 para banderas invalidas.

Convertir estos "0.0" a NaN

In [285]:
met = rm_invalid_flags(met)
tmp = met[met['Flag'].isin(invalid_flags)]
print(f"N. filas con bandera invalida: {len(tmp)}")
print(f"N. filas con bandera invalida y con Nan: {tmp['Value'].isna().sum()}")

N. filas con bandera invalida: 5165
N. filas con bandera invalida y con Nan: 5165


Encontrar el número de valores NaN con bander valida

In [286]:
tmp = met[~met['Flag'].isin(invalid_flags)]
print(f"Total NaN de banderas validas \n{tmp.isna().sum().sum()}")

Total NaN de banderas validas 
1828


Faltantes por variable

In [287]:
piv_met = pivot(met)
piv_met = piv_met.reset_index()
print(piv_met.isna().sum())
print(f"Total faltantes: {piv_met.isna().sum().sum()}")

parameter
date        0
PRS       214
RAINF     211
RH        233
SR       5152
TOUT      214
WDR       704
WSR       268
dtype: int64
Total faltantes: 6996


* La mayoría de estos NaN se encuentran en Radiación Solar

In [288]:
met.to_csv(PROCESSED_MET_PATH, index=False)
piv_met.to_csv(PROCESSED_PIV_MET_PATH, index=False)